# 時系列データ用ノイズ検出モデルの学習（Google Colab版）

このノートブックは、時系列データを用いた30クラス分類によるノイズ検出モデルの学習を行います。

## 📋 セットアップ手順

1. **GPUを有効化**: ランタイム → ランタイムのタイプを変更 → GPU（T4）を選択
2. **Google Driveをマウント**
3. **データセットの準備**: 先に `prepare_dataset_timeseries.ipynb` を実行して `baseline_dataset_timeseries.pickle` を生成
4. 各セルを順番に実行してください


## 1. Google Driveをマウント


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


## 2. ライブラリのインストールとインポート


In [ ]:
# ライブラリのインストール
!pip install torch torchvision scikit-learn matplotlib -q

import pickle
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
import platform
import os
import sys
import time
import warnings

warnings.filterwarnings('ignore', category=UserWarning, module='matplotlib')

# GPUの確認
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"使用デバイス: {device}")
if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"GPUメモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

# 日本語フォントの設定（Colab用）
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['axes.unicode_minus'] = False


## 3. 作業ディレクトリの設定


In [ ]:
# 作業ディレクトリを設定
project_root = '/content/drive/MyDrive/soturon'  # soturonフォルダのルート
baseline_folder = f'{project_root}/畳み込み'  # 畳み込みフォルダ

if os.path.exists(baseline_folder):
    os.chdir(baseline_folder)
    sys.path.insert(0, baseline_folder)
    sys.path.insert(0, project_root)
    print(f"Working directory: {os.getcwd()}")
else:
    print(f"⚠️ {baseline_folder} が見つかりません。")
    print("Google Driveにsoturonフォルダを作成して、ファイルをアップロードしてください。")


## 4. 必要なモジュールのインポート


In [ ]:
# モデルとユーティリティをインポート
from model import SimpleCNN
from utils.evaluate import evaluate_baseline_model, plot_confusion_matrix
from train_timeseries import TimeSeriesDataset, train_epoch, validate

print("✅ モジュールのインポートに成功しました！")


## 5. データセットの読み込み


In [ ]:
# データセットの読み込み
dataset_path = f'{baseline_folder}/baseline_dataset_timeseries.pickle'

if os.path.exists(dataset_path):
    print(f"✅ データセットファイルが見つかりました: {dataset_path}")
    
    with open(dataset_path, 'rb') as f:
        dataset = pickle.load(f)
    
    train_data = dataset['train']['data']
    train_labels = dataset['train']['labels']
    val_data = dataset['val']['data']
    val_labels = dataset['val']['labels']
    test_data = dataset['test']['data']
    test_labels = dataset['test']['labels']
    
    config = dataset.get('config', {})
    print(f"\nデータセットの情報:")
    print(f"  訓練データ: {len(train_data):,}サンプル")
    print(f"  検証データ: {len(val_data):,}サンプル")
    print(f"  テストデータ: {len(test_data):,}サンプル")
    print(f"  データタイプ: {config.get('data_type', 'unknown')}")
    print(f"  ノイズタイプ: {config.get('noise_type', 'unknown')}")
    print(f"  ノイズレベル: {config.get('noise_level', 'unknown')}")
else:
    print(f"⚠️ データセットファイルが見つかりません: {dataset_path}")
    print("先に prepare_dataset_timeseries.ipynb を実行して、データセットを準備してください。")


## 6. ハイパーパラメータの設定


In [ ]:
# ハイパーパラメータ
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 10
NUM_CLASSES = 30

print(f"ハイパーパラメータ:")
print(f"  バッチサイズ: {BATCH_SIZE}")
print(f"  学習率: {LEARNING_RATE}")
print(f"  エポック数: {NUM_EPOCHS}")
print(f"  クラス数: {NUM_CLASSES}")


## 7. DataLoaderの作成


In [ ]:
# DataLoaderの作成
train_dataset = TimeSeriesDataset(train_data, train_labels)
val_dataset = TimeSeriesDataset(val_data, val_labels)
test_dataset = TimeSeriesDataset(test_data, test_labels)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

print("✅ DataLoaderの作成が完了しました")


## 8. モデルの作成


In [ ]:
# モデルの作成
model = SimpleCNN(num_classes=NUM_CLASSES).to(device)
print(f"モデルパラメータ数: {sum(p.numel() for p in model.parameters()):,}")

# モデルの初期化を確認
model.eval()
with torch.no_grad():
    dummy_input = torch.randn(1, 3000).to(device)
    dummy_output = model(dummy_input)
    print(f"  ダミー入力の形状: {dummy_input.shape}")
    print(f"  ダミー出力の形状: {dummy_output.shape}")
    print(f"  ダミー出力の範囲: [{dummy_output.min().item():.4f}, {dummy_output.max().item():.4f}]")
model.train()


## 9. 損失関数とオプティマイザの設定


In [ ]:
# 損失関数とオプティマイザ
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

print("✅ 損失関数とオプティマイザの設定が完了しました")


## 10. 学習の実行


In [ ]:
# 学習履歴
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []
best_val_acc = 0
best_model_state = None

print("学習開始...")
print("=" * 60)
print(f"総エポック数: {NUM_EPOCHS}")

start_time = time.time()

for epoch in range(NUM_EPOCHS):
    epoch_start = time.time()
    
    # 学習
    train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer, device)
    
    # 検証
    val_loss, val_acc, _, _ = validate(model, val_loader, criterion, device)
    
    # 学習率の調整
    scheduler.step(val_loss)
    
    # 履歴の保存
    train_losses.append(train_loss)
    train_accuracies.append(train_acc)
    val_losses.append(val_loss)
    val_accuracies.append(val_acc)
    
    # ベストモデルの保存
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_model_state = model.state_dict().copy()
    
    # ログ出力
    if (epoch + 1) % 5 == 0 or epoch == 0:
        epoch_time = time.time() - epoch_start
        elapsed_time = time.time() - start_time
        remaining_epochs = NUM_EPOCHS - (epoch + 1)
        estimated_remaining = (elapsed_time / (epoch + 1)) * remaining_epochs if epoch > 0 else 0
        
        print(f"Epoch [{epoch+1}/{NUM_EPOCHS}] ({epoch_time:.1f}秒)")
        print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
        print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%")
        if epoch > 0:
            print(f"  経過時間: {elapsed_time/60:.1f}分, 残り見積もり: {estimated_remaining/60:.1f}分")
        print()

total_time = time.time() - start_time
print(f"学習完了！総時間: {total_time/60:.1f}分 ({total_time/3600:.2f}時間)")
print(f"ベスト検証精度: {best_val_acc:.2f}%")


In [ ]:
# ベストモデルをロード
model.load_state_dict(best_model_state)

# テストデータで評価
print("\nテストデータで評価中...")
test_loss, test_acc, test_predictions, test_labels_list = validate(
    model, test_loader, criterion, device
)

print(f"テスト精度: {test_acc:.2f}%")

# 詳細な評価
results = evaluate_baseline_model(test_predictions, test_labels_list)
print(f"\n詳細な評価結果:")
print(f"  Accuracy: {results['accuracy']:.4f}")
print(f"  Precision: {results['precision']:.4f}")
print(f"  Recall: {results['recall']:.4f}")
print(f"  F1-score: {results['f1_score']:.4f}")


## 12. 結果の可視化


In [ ]:
# 混同行列を可視化
plot_confusion_matrix(
    results['confusion_matrix'],
    title='混同行列（テストデータ、時系列データ）',
    save_path=None  # Colabでは表示のみ
)
plt.show()

# 学習曲線を可視化
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].plot(train_losses, label='訓練損失', color='blue')
axes[0].plot(val_losses, label='検証損失', color='red')
axes[0].set_xlabel('エポック')
axes[0].set_ylabel('損失')
axes[0].set_title('学習曲線（損失、時系列データ）')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

axes[1].plot(train_accuracies, label='訓練精度', color='blue')
axes[1].plot(val_accuracies, label='検証精度', color='red')
axes[1].set_xlabel('エポック')
axes[1].set_ylabel('精度 (%)')
axes[1].set_title('学習曲線（精度、時系列データ）')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 13. モデルの保存


In [ ]:
# モデルをGoogle Driveに保存
model_save_path = f'{baseline_folder}/baseline_model_timeseries.pth'
torch.save({
    'model_state_dict': model.state_dict(),
    'test_accuracy': test_acc,
    'results': results,
    'config': config,
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies,
}, model_save_path)

print(f"✅ モデルを保存しました: {model_save_path}")
print(f"   テスト精度: {test_acc:.2f}%")
print(f"   Accuracy: {results['accuracy']:.4f}")
print(f"   F1-score: {results['f1_score']:.4f}")
